Note: Student name removed. Submitted, Fall 2019.

## 1. Visualization technique
The techniques I plan to use are: <br>
*Combination charts (a mix of bar and line plots).<br>
*Simple line chart.<br>

<font color=blue> --Details about this technique:  </font><br>
Both bar plots and line plots are excellent techniques to use when showing data over *time*.  Either method <br> 
can easily show the user how the data has *increased* or *decreased* over time.  <br>
With bar charts, it's easier to see changes when they are larger.  But depending on the application, bar charts <br> can still be useful to help the user see consistency.  In the case of this visualization, I **do** care about <br> 
the changes, even if they are subtle, so enter the line plot, thus creating a combo chart. <br> 
The line chart can plot the %changes of the bars.  With the addition of the line chart, the user can quickly <br> 
see how much the data has changed from period to period.<br>

Bar and line plots are ***not*** good for showing distrubutions, means, medians, modes, or outliers. <br>
When that type of analysis is more insightful or requested, better techniques for visualization include <br>
creating box and whisker plots, and/or violin plots.  



## 2. Visualization Library
The library I'm using is Plotly.

Why I chose it:  <br>
To my surprise, I couldn't find options for creating combo charts.  This includes Seaborn, Bokeh, and Altair. <br>
What I like about Plotly is it's pretty easy to use and understand.  My goal is to create insightful data <br>
by using the absolute *simplest* method possible.  Plotly provides a handful of nice features that are easy to <br>
implement on these bar and line plots, so that is why this library was used.  I can simply create bars and <br>
lines together, and I can simply add hovering data labels, or fixed labels.  This is exactly what I want to show a user.<br>

<font color=blue>--Details on Plotly.</font><br>
The way Plotly works is it has a JSON object under every graph.  Hopefully JSON will be understandable to a <br>
designer or user if XML was ever used. The great thing about JSON is it's a dictionary data structure, which <br> python users will be familiar with.  The documentation also explains that Plotly graphs are interactive, so <br>
users can inspect data by zooming, selecting, and/or hovering over data. <br>

Plotly is a company founded in Montreal, Cananda. This is another reason I went with Plotly.  I'm learning <br>
a great deal about Python, Pandas, and programming from a certain Canadian professor, so I thought I would <br>
keep it rolling with Plotly.  It was founded by Alex Johnson, Jack Parmer, Chris Parmer, and Matthew Sundquist. <br>
The company does create products that are open source, such as plotly.py, which is built on top of their  <br> JavaScript library.  The charts and plots can be displayed directly in Jupyter notebooks.

<font color=blue>To install:</font><br>
Go to the command line and type: <br>
$ pip install plotly==4.3.0



# 3. Demonstration

My goal is to help investors understand characteristics of businesses.  <br> 
To quote Einstein: <br>
“Compound interest is the 8th wonder of the world. He who understands it, earns it; he who doesn't, pays it.”  <br>
I sincerely believe this.  <br>
<br>
To help investors, I would like them to understand basic stats on business.  <br> 
I find parallels with looking at baseball cards. <br>
In my younger days, I couldn't wait to look at the back and start understanding averages, hits, singles, doubles, <br>
triples,  homeruns, walks, and, strikeouts, and how those numbers changed year-over-year.  With investing, <br> 
my goal is to understand basic numbers in similar fashion.  I don't believe anyone should invest in a <br> 
company if they don't understand how <br> basic numbers have changed over time, let alone if they don't know <br> 
the numbers at all.<br>
With that, we will look at Apple, Inc.

In [19]:
#Import all librabries 
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
#import plotly.graph_objects as go
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from matplotlib import rcParams
#import plotly.chart-studio as py


## Download Income Statement and Balance Sheet for Apple from Morningstar.com.

<font color=blue>To get dataset: </font><br>
Go to morningstar.com, type in "AAPL" in the search box.<br>
Hit Enter-<br>
Click "Financials" - you will get basic data on this screen.<br>
Click "Income Statement" - now you will see 5 years on data.  Click on "export to excel" <br>
Save this data in a directory you can access.<br>
You can repeat steps for both the "Balance Sheet", and "Statement of Cash Flows."<br>
I did not use "Statement of Cash Flows" for this demonstration. <br>
I have renamed the Income statement as: <font color=blue>"aapl_Income Statement.xls"</font>, <br> 
 and Balance Sheet as: <font color=blue>"aapl_Balance Sheet.xls"</font>    

In [20]:
def get_income_statement():
    Inc_Statement = pd.read_excel('aapl_Income_Statement.xls', usecols = "A:G")
    #To clean dataset:
    # We will be looking at Revenue (Rev), Net Income (NI), and Profit Margin (calculation - NI/Rev) 
    # We will also look at how different margins behave to understand how cost might be affecting Net Income.
    # We will look at Gross Margin (calculation - Gross Profit/Rev), Operating Margin (calculation - Operating Profit/Rev)
    # and Profit Margin again (calculation - NI/Rev).
    # Below we are pulling just those fields needed for the amounts and calcs from the income statement.
    df_IS_base = Inc_Statement[(Inc_Statement['Name'].str.strip()=='Total Revenue') | 
                     (Inc_Statement['Name'].str.strip()=='Gross Profit') |
                     (Inc_Statement['Name'].str.strip()=='Total Operating Profit/Loss') |
                     (Inc_Statement['Name'].str.strip()=='Net Income Available to Common Stockholders') ]
    df_T = df_IS_base.T
    #getting rid label row
    df_is = df_T[df_T[1].str.strip()!='Total Revenue']
    df_is = df_is.reset_index()
    #getting rid of Trailing 12 month data from MorningStar
    df_is = df_is[df_is['index'].str.strip()!='TTM']
    df_is
    #renaming columns - this was done using column position to handle processing on macs and PCs.  
    #I noticed row differences depending on the system.
    df_is = df_is.rename(columns={df_is.columns[0]:'Year',
                                 df_is.columns[2]:'Revenue',
                                 df_is.columns[1]:'Gross_Profit',
                                 df_is.columns[3]:'Oper_Income',
                                 df_is.columns[4]:'Net_Income'})

    #remove commas from string, and convert to float.
    df_is["Rev_num"] = df_is["Revenue"].str.replace(",","").astype(float)
    df_is["GI_num"] = df_is["Gross_Profit"].str.replace(",","").astype(float)
    df_is["OI_num"] = df_is["Oper_Income"].str.replace(",","").astype(float)
    df_is["NI_num"] = df_is["Net_Income"].str.replace(",","").astype(float)

    #calculate YOY change for Revenue
    df_is["Rev_perc_chg"] = df_is["Rev_num"].pct_change(-1)
    #using top 4 rows, because we don't have percent change for all 5 years.  We would need 6 years to show 5 YOYs.
    df_is = df_is.head(4)
    #calculate Margins
    df_is['Gross_Margin'] = df_is["GI_num"]/df_is["Rev_num"]
    df_is['Operating_Margin'] = df_is["OI_num"]/df_is["Rev_num"]
    df_is['Profit_Margin'] = df_is["NI_num"]/df_is["Rev_num"]
    df_is
    #put data needed for charts into lists
    xvals_yr = df_is['Year'].to_list()
    revenue = df_is['Rev_num'].to_list()
    revenue_perc = df_is['Rev_perc_chg'].to_list()
    net_income = df_is['NI_num'].to_list()
    gross_marg = df_is['Gross_Margin'].to_list()
    operating_marg = df_is['Operating_Margin'].to_list()
    profit_marg = df_is['Profit_Margin'].to_list()
    
    return df_is

In [21]:
def rev_growth():

    # Show Revenue: Create Plotly figure with secondary y-axis (Bar-plot with line-plot on secondary axis)
    xvals = ['2019','2018','2017','2016']
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Bar(x=xvals, y=revenue, name="Revenue", 
           #hovertemplate='$%{y:.0f}',
           text=revenue, 
           textposition='auto',
           #texttemplate='$%{y:.0f}'
           texttemplate='$%{y:$,}'
           #texttemplate = "%{label}"
           #texttemplate='$%{y:.3s}'
           #texttemplate='$%{y:.1s}'
           #texttemplate='$%{y:.04d}'
           #textformat='auto'
           #textformat=','
           ),
        secondary_y=False,
        )

    fig.add_trace(
        go.Scatter(x=xvals, y=revenue_perc, name="% change",
              #text=revenue_perc, 
              #textposition='line',
              #texttemplate='$%{y:.0f}'
              #texttemplate='$%{y:%}'
              ),
        secondary_y=True,
        )

    # Create figure title
    fig.update_layout(
        title_text="Figure 1: Revenue Growth - Apple, Inc."
        )

    # x-axis title
    fig.update_xaxes(title_text="Year")

    # y-axes titles
    fig.update_yaxes(title_text="<b>primary</b> Revenue ($)", secondary_y=False)
    fig.update_yaxes(title_text="<b>secondary</b> YOY change (%)", secondary_y=True)
    fig.update_layout(yaxis2_tickformat = '%')
    #fig.update_layout(yaxis_tickformat = ',')

    # update x-axis values, and secondary y-axis range
    fig.update_layout(
        xaxis = dict(
            tickmode = 'array',
            tickvals = [2019, 2018, 2017, 2016],
            ticktext = ['2019', '2018', '2017', '2016']
                    ),
        yaxis2 = dict(
            #tickmode='linear',
            range=[-.10, .60],
            side='right'
            )
    
        )

    fig.show()

rev_data = get_income_statement()
revenue = rev_data['Rev_num'].to_list()
revenue_perc = rev_data['Rev_perc_chg'].to_list()
rev_growth()


### From Figure 1:
You see that Apple has **Revenue** of: <br> 
215B, 229B, 265B and 260B in Revenue from 2016-2019<br><br> 
First off, I want you to understand this number. When you think Apple, you should be able to say they <br>
had 260 Billion in revenue, just like knowing Tony Gywnn hit .324 in 2001.<br>
What has happened to revenue over time?  It decreased by 5 billion in Fiscal Year 2019.  Why?  <br>
The bars show the revenue actuals, while the line shows the percent change.  We had a nice 15% increase in <br>
2018, followed up by a 2% decrease, or 5 billion, in Fiscal Year 2019.  Why?<br><br>
Is it because of they had product problems, supplier problems, a down economy, competition pressure, <br>
regulatory pressure, or foreign exchange loss?  The point is you should start to understand why these numbers <br>
are changing.  It's just like looking at the average in baseball.  Is the player getting older, is he injured, <br>
have pitchers figured him out, did the surrounding lineup change, is the defense shifting on him? <br><br>
With the flucuation in revenue, we should ask, how does this affect **Net Income?** <br>
We will have to dig into the financial statements to understand what is going on.

In [22]:
def get_profit_margin():
    xvals = ['2019','2018','2017','2016']
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Bar(x=xvals, y=revenue, name="Revenue"),
        secondary_y=False,  
        )

    fig.add_trace(
        go.Bar(x=xvals, y=net_income, name="Net Income"),
        secondary_y=False,
        )

    fig.add_trace(
        go.Scatter(x=xvals, y=profit_marg, name="Profit Margin"),
        #xaxis=dict(showgrid=False, zeroline=False),
        #yaxis=dict(showgrid=False, zeroline=False),
        secondary_y=True,
        )

    # Create figure title
    fig.update_layout(
        title_text="Figure 2: Profit Margin (Rev / Net Income) - Apple, Inc."
        )

    # x-axis title
    fig.update_xaxes(title_text="Year")

    # y-axes titles
    fig.update_yaxes(title_text="<b>primary</b> Rev & Net Income ($)", secondary_y=False)
    fig.update_yaxes(title_text="<b>secondary</b> Profit Margin(%)", secondary_y=True)
    fig.update_layout(yaxis2_tickformat = '%')
    # update x-axis values
    fig.update_layout(
        xaxis = dict(
            tickmode = 'array',
            tickvals = [2019, 2018, 2017, 2016],
            ticktext = ['2019', '2018', '2017', '2016'],        
        )
    )
    #update second y-axis range
    fig.update_layout(
        yaxis2=dict(
            #tickmode='linear',
            range=[0, .40],
            side='right'
        )
    
    )

    fig.show()

rev_data = get_income_statement()
revenue = rev_data['Rev_num'].to_list()
net_income = rev_data['NI_num'].to_list()
profit_marg = rev_data['Profit_Margin'].to_list()
get_profit_margin()

### From Figure 2:
You see that Apple has consistent Revenue, and Net Income, thus making Profit Margin also consistent over time. <br>
There was a mild change in 2018, which pushed profit margin up slightly to 22%, vs. 21% in other years. <br>
This means somthing in the cost structure was favorable in 2018.  This is good news.  Any time we are making <br>
more profit on revenue is a good thing.  What we really want to look out for is detecting **decreases.**  <br>
We want to understand what in the cost structure might be eating up our profits. <br><br>

Now let's compare **different margins** to understand what section of the income statement might indicate where <br>
profit is impacted.

In [23]:
# Show Margins: Create Plotly figure line plots
def get_margins():
    xvals_02 = ['2019','2018','2017','2016']

    fig_02 = make_subplots(specs=[[{"secondary_y": False}]])

    # Add traces
    fig_02.add_trace(
        go.Scatter(x=xvals_02, 
            y=df_gm, 
            name="Gross Margin"),
            secondary_y=False,
          )

    fig_02.add_trace(
        go.Scatter(x=xvals_02, 
            y=df_op, 
            name="Operating Margin"),
            secondary_y=False,
         )

    fig_02.add_trace(
        go.Scatter(x=xvals_02, 
            y=df_pm, 
            name="Profit Margin"),
            secondary_y=False,
         )
    # Add figure title
    fig_02.update_layout(
        title_text="Figure 3: Margin Comparison - Apple, Inc."
        )

    # Set x-axis title
    fig_02.update_xaxes(title_text="Year")

    # Set y-axes titles
    fig_02.update_yaxes(title_text="Percentage (%)", secondary_y=False)
    #fig_02.update_yaxes(title_text="<b>secondary</b> change (%)", secondary_y=True)
    fig_02.update_layout(yaxis_tickformat = '%')

    fig_02.update_layout(
        xaxis = dict(
        #tickmode = 'array',
        tickvals = [2019, 2018, 2017, 2016],
        ticktext = ['2019', '2018', '2017', '2016']       
        )
    )

    fig_02.update_layout(
        yaxis=dict(
        #tickmode='linear',
        range=[0, .50]
        )
   
    )

    fig_02.show()
    
#Get Margin data from Income Statement
rev_data = get_income_statement()    
df_margins = rev_data[['Gross_Margin','Operating_Margin','Profit_Margin']]
df_gm = df_margins['Gross_Margin'].to_list()
df_op = df_margins['Operating_Margin'].to_list()
df_pm = df_margins['Profit_Margin'].to_list()
get_margins()

### From Figure 3:
The breakout of margins tell us a little more. <br>
Gross Margin is consistent. This is the <font color=blue> blue </font> line above. <br>
The margin has gone down 1%, telling us cost of goods sold has increased very slightly. <br><br>
Operating margin, however has gone down from 28% to 25%.  This is a decent decrease. <br>
This is the <font color=red> red </font> line above.
Line items affecting operating margin are selling, general, and administrative expenses, as well as R&D. <br>
Why did we see a drop like this?  Did Apple increase R&D spending? They certainly need R&D, but are they <br>
feeling the competition to the iPhone?  Did they increase salaries?  This is worth looking into. <br><br>
Profit Margin has been pretty consistent at around 21%.  This is the <font color=green> green </font> line above. <br>
Items affecting this margin in this view are interest and taxes.  We don't see anything immediately, <br>
but recent tax laws should favorably benefit Apple's bottom line.  <br><br>
Looking at margins like this let us further break down how profits are impacted.


One last metric we can look at for understanding the business, is looking at **capital structure**.

In [24]:
def get_balance_sheet():
    Bal_Sheet = pd.read_excel('aapl_Balance_Sheet.xls', usecols = "A:G")
    Bal_Sheet
    Bal_Sheet = Bal_Sheet[(Bal_Sheet['Name'].str.strip()=='Total Assets') |
                  (Bal_Sheet['Name'].str.strip()=='Total Liabilities') |
                  (Bal_Sheet['Name'].str.strip()=='Total Equity')]

    df_bs = Bal_Sheet.T
    #Test02.reset_index()
    df_bs = df_bs[df_bs[0].str.strip()!='Total Assets']
    df_bs = df_bs.reset_index()
    df_bs
    df_bs = df_bs[df_bs['index'].str.strip()!='TTM']
    df_bs
    df_bs = df_bs.rename(columns={df_bs.columns[1]:'Assets',
                                    df_bs.columns[2]:'Liabilities',
                                    df_bs.columns[3]:'Equity'
                                           })
  
    df_bs
    df_bs['Asset_num'] = df_bs['Assets'].str.replace(",", "").astype(float)
    df_bs['Liab_num'] = df_bs['Liabilities'].str.replace(",", "").astype(float)
    df_bs['Equity_num'] = df_bs['Equity'].str.replace(",", "").astype(float)
    df_bs['Debt_perc'] = df_bs['Liab_num'] / df_bs['Asset_num']
    df_bs['Equity_perc'] = df_bs['Equity_num'] / df_bs['Asset_num']
    
    return df_bs


In [25]:
def get_cap_structure():

    x_vals_04=['2019', '2018', '2017', '2016', '2015']
    plt.close
    fig_04 = go.Figure(data=[ 
        go.Bar(name='Debt', x=x_vals_04, y=debt_perc),
        go.Bar(name='Equity', x=x_vals_04, y=equity_perc)
        ])
    # Change the bar mode
    fig_04.update_layout(title='Figure 4: Debt vs. Equity - Apple, Inc.', barmode='stack')
    fig_04.update_layout(yaxis_tickformat = '%')
    fig_04.show()

bs_data = get_balance_sheet() 
equity_perc = bs_data['Equity_perc'].to_list()
debt_perc = bs_data['Debt_perc'].to_list()
get_cap_structure()

### From Figure 4:
This is somewhat surprising seeing the change over time.  Apple was 59% debt and 41% equity in 2015, <br>
but it's increased to 73% debt and 27% equity in 2019.  Tech companies usually are financed with <br>
equity, and use their cash for R&D purposes.  Generally, we see more debt with companies that are older <br>
and mature and have stable cash flows.  Apple certainly has stable cash flows, but they have such a giant <br>
cash position, that they could finance the business with cash. This would of course compete with R&D spend. <br>
Apple could also be taking advantage of favorable borrowing rates, but again why borrow, when you have mountains <br>
of cash.  It's quite interesting, and we could dig into their reasoning on why they think this is a good idea.  <br>
In general, you like to see a company have less debt obligation, because down cycles are a certainty of business. <br><br>

So this was a very basic walkthough.  There is so much more we can dig into.  But the basic idea is I want <br>
investors to understand the numbers behind a business before they start buying positions.  You should think <br>
of these numbers like kids do for sports cards.  You should understand how big the business is, what direction <br>
net income is heading, and what pressure is on profits at the very minimum.  <br><br>

And even after getting a decent grasp on the numbers, we still need to determine if the stock is actually *priced*<br>
at a value worth buying.  That's where the fun comes in, and I hope to dig into another visualization with an <br>
example on my next project.





# 4.  Rule et al's rules for computational analyses
I specifically attempted to implement these rules in my visualization development: <br>
Rule 1: Tell a story for an audience <br>
Rule 2: Document the process, not just the results <br>
Rule 3: Use cell divisions to make steps clear <br>
Rule 4: Modularize code <br>